Importing libraries

In [1]:
import sympy as sp
import numpy as np

**Question 1** - Approximate sin(2x) with using the first five terms of the Taylor series expansion for x=π/4.
Explain what type of error we are dealing with for this problem. Calculate absolute error,
relative error and percentage error. 


In [2]:
def taylor_series(func, x, x0, n_term, x_value, x0_value):
    """
    :param func: Function to approximate with taylor series
    :param x: Symbol using in the function. E.g. X for sin(X)
    :param x0: Symbol using in the function.
    :param n_term: Number of terms, e.g. first n terms
    :param x_value: Value of x
    :param x0_value: Referance point
    :return: Approximate value
    """

    sum = 0

    x_x0 = (x - x0)

    for i in range(n_term):
        fx0 = func.evalf(subs={x: x0_value})
        x_x0_n = (x_x0.evalf(subs={x: x_value, x0: x0_value})) ** i
        fact_n = sp.factorial(i)
        # f(x0) * (x-x0)^n / fact(n)
        sum += fx0 * x_x0_n / fact_n
        func = sp.diff(func, x)

    return sum

In [3]:
# We have to create x and a as "symbol" which are parameters
# of tylor series expansion
x = sp.Symbol('x')
x0 = sp.Symbol('x0')
# Create function sin(2x)
f = sp.sin(2 * x)
# Set x = pi/4 and a = pi/4
x0_value = 0
x_value = sp.pi / 4
# First 5 term is tylor series will be used
n_steps = 5

# Our function result
approximate_value = taylor_series(f, x, x0, n_steps, x_value, x0_value)
# Real sin(2*pi/4) result
exact_value = f.evalf(subs={x: x_value})

print("Approximate value: ", approximate_value)
print("Exact value:       ", exact_value)

absolute_error = np.abs(exact_value - approximate_value)
relative_error = absolute_error / np.abs(exact_value)
percentage_error = relative_error * 100

print("Absolute error: ", absolute_error)
print("Relative error: ", relative_error)
print("Percentage error: ", percentage_error)

Approximate value:  0.924832229288650
Exact value:        1.00000000000000
Absolute error:  0.0751677707113496
Relative error:  0.0751677707113496
Percentage error:  7.51677707113496


**Question 2** - Use Gauss elimination to solve the equations Ax=B

In [4]:
def gauss_elimination(A, b):
    """
    :return: x vector
    """
    n = len(b)
    x = np.zeros(n, float)
    # Create and use copies of A matrix and b vector because their values
    # will be changed during calculation.
    a_copy = np.copy(A)
    b_copy = np.copy(b)

    for k in range(n - 1):
        for i in range(k + 1, n):
            if a_copy[i, k] == 0:
                continue
            factor = a_copy[k, k] / a_copy[i, k]
            for j in range(k, n):
                a_copy[i, j] = a_copy[k, j] - a_copy[i, j] * factor
            b_copy[i] = b_copy[k] - b_copy[i] * factor

    x[n - 1] = b_copy[n - 1] / a_copy[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        sum_ax = 0
        for j in range(i + 1, n):
            sum_ax += a_copy[i, j] * x[j]
        x[i] = (b_copy[i] - sum_ax) / a_copy[i, i]

    return x

In [5]:
# Create A matrix
A = np.array([[2, -3, -1],
              [3, 2, -5],
              [2, 4, -1]], dtype=np.float)
# Create b vector
b = np.array([3, -9, -5], dtype=np.float)

# Calculate x
x = gauss_elimination(A, b)

print("x :", x)

# Correction
print("Correction\nA.x is: ", np.matmul(A, x), " and b :", b)

x : [ 0.65306122 -1.14285714  1.73469388]
Correction
A.x is:  [ 3. -9. -5.]  and b : [ 3. -9. -5.]


**Question 3** - Solve the equations Ax=B by Doolittle’s decomposition method

In [6]:
def doolittle_decomposition(A, b):
    #  Initialize L and U
    n = len(A)
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    # Fill L and U with their values
    for j in range(n):
        L[j][j] = 1

        for i in range(j + 1):
            sum_ = sum(U[k][j] * L[i][k] for k in range(i))
            U[i][j] = A[i][j] - sum_

        for i in range(j, n):
            sum_ = sum(U[k][j] * L[i][k] for k in range(j))
            try:
                L[i][j] = (A[i][j] - sum_) / U[j][j]
            except ZeroDivisionError:
                pass

    print("L:\n", L)
    print("U:\n", U)

    """   Find Y from L.Y = b with gauss elimination   """
    Y = gauss_elimination(L, b)
    print("\ny :", Y)

    """   Find x from U.x = Y with gauss elimination   """
    x = gauss_elimination(U, Y)

    return x

In [7]:
# Create A matrix
A = np.array([[2.34, -4.1, 1.78],
              [-1.98, 3.47, -2.22],
              [2.36, -15.17, 6.18]], dtype=np.float32)

# Create b vector
b = np.array([0.02, -0.73, -6.63], dtype=np.float32)

# Calculate x
x = doolittle_decomposition(A, b)

print("\nx :", x)
# correction
print("\nCorrection\nA.x : ", np.matmul(A, np.transpose(x)), " and b: ", b)

L:
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-8.46153885e-01  1.00000000e+00  0.00000000e+00]
 [ 1.00854700e+00 -1.43464027e+04  1.00000000e+00]]
U:
 [[ 2.33999991e+00 -4.09999990e+00  1.77999997e+00]
 [ 0.00000000e+00  7.69179405e-04 -7.13846137e-01]
 [ 0.00000000e+00  0.00000000e+00 -1.02367393e+04]]

y : [ 1.99999996e-02 -7.13076932e-01 -1.02367390e+04]

x : [1.00000587 1.00000333 0.99999997]

Correction
A.x :  [ 0.02       -0.73000001 -6.63003722]  and b:  [ 0.02 -0.73 -6.63]


**Question 4** - Solve the equations Ax=B by Choleski’s decomposition method

In [8]:
def cholosky(matrix):
    # initialize lower matrix with same dimensions of input matrix
    Lower = np.zeros((matrix.shape[0], matrix.shape[1]))
    n, _ = np.shape(matrix)
    for j in range(n):
        for i in range(j, n):
            if i == j:
                Lower[i, j] = np.sqrt(matrix[i, j]-np.sum(Lower[i, :j]**2))
            else:
                Lower[i, j] = (matrix[i, j]-np.sum(Lower[i, :j]*Lower[j, :j])) / Lower[j, j]
    return Lower

In [9]:
# Create A matrix
A = np.array([[1, 1, 1],
            [1, 2, 2],
            [1, 2, 3]], dtype=np.float32)

# Create b vector
b = np.array([1, 1.5, 3], dtype=np.float32)

print("A:\n", A, "\nb:", b)

# Find L and L transpose
L = cholosky(A)
L_transpose = np.transpose(L)

"""
A.x = b
L.L_transpose.x = b
L_transpose.x = Y
L.Y = b

first find Y from L.Y = b
then find x from L_transpose.x = Y
"""

# Find Y from L.Y = b
Y = gauss_elimination(L, b)
# Find x from L_transpose.x = Y
x = gauss_elimination(L_transpose, Y)

print("\nx:", x)
print("\nCorrection A.x :\n", np.matmul(A, x), "and b : ", b)

A:
 [[1. 1. 1.]
 [1. 2. 2.]
 [1. 2. 3.]] 
b: [1.  1.5 3. ]

x: [ 0.5 -1.   1.5]

Correction A.x :
 [1.  1.5 3. ] and b :  [1.  1.5 3. ]
